In [110]:
from typing import TypedDict, Annotated, List, Literal, Dict, Any 
from langchain_core.messages import BaseMessage, HumanMessage, AIMessage, SystemMessage
from langgraph.graph import StateGraph, END, MessagesState 
from langchain_groq import ChatGroq 
from langchain_core.prompts import ChatPromptTemplate, PromptTemplate
from dotenv import load_dotenv 
from langchain_classic.agents import initialize_agent, AgentType
from langchain_tavily import TavilySearch
from langchain_community.tools.arxiv import ArxivQueryRun
from langchain_community.utilities import ArxivAPIWrapper 
from langgraph.types import Interrupt

In [111]:
load_dotenv()

True

In [112]:
llm = ChatGroq(model = "llama-3.3-70b-versatile")

In [113]:
query_input = input("Please enter your topic: ")

In [114]:
query_input

'what is attention is all youu need in nlp?'

In [115]:
class SupervisorState(MessagesState):
    """State for the multi-agent"""
    next_agent: str = "supervisor"
    web_data: str = ""
    arxiv_data: str = ""
    final_report: str = ""

In [116]:
def supervisor_agent(state: SupervisorState) -> Dict:
    has_web_data = bool(state.get("web_data"))
    has_arxiv_data = bool(state.get("arxiv_data"))
    has_final_report = bool(state.get("final_report"))

    if not has_web_data and not has_arxiv_data and not has_final_report:
        next_agent = "web_search"
    
    elif has_web_data and not has_arxiv_data and not has_final_report:
        next_agent = "writer"

    elif has_web_data and has_final_report:
        user_input = input(prompt = "Do you want a deeper answer? [y/n]")
        if user_input == "y":
            next_agent = "arxiv_search"
        else:
            next_agent = "__end__"
        
    if has_web_data and has_arxiv_data and has_final_report:
        next_agent = "__end__"

    return {
        "next_agent": next_agent
    }

In [117]:
def web_agent(state: SupervisorState) -> Dict:
    """Gathers data from web"""
    b = []
    messages = state.get("messages", [])
    user_message = ""
    for m in messages:
        if isinstance(m, HumanMessage):
            user_message = m.content
    tavily = TavilySearch()
    response = tavily.invoke({"query": user_message})
    for item in response.get("results", []):
        b.append(item.get("content", ""))
    
    return {
        "web_data": b,
        "next_agent": "supervisor"
    }

In [118]:
def arxiv_agent(state: SupervisorState) -> Dict:
    messages = state.get("messages", [])
    user_message = ""
    for m in messages:
        if isinstance(m, HumanMessage):
            user_message = m.content

    wrapper = ArxivAPIWrapper(top_k_results=5)
    arxiv = ArxivQueryRun(api_wrapper=wrapper)
    result = arxiv.invoke(user_message)

    return {
        "arxiv_data": result,
        "stage": "after_arxiv",
        "next_agent": "supervisor"
    }

In [119]:
def writer_agent(state: SupervisorState) -> Dict:
    """write a report"""
    message = state.get("messages")
    for m in message:
        if isinstance(m, HumanMessage):
            question = m.content

    web_data = state.get("web_data", "")
    arxiv_data = state.get("arxiv_data", "")

    writing_prompt = f"""
Answer the question based only on the text below:

question: {question}

text: {web_data} | {arxiv_data}
"""

    report_response = llm.invoke([HumanMessage(content = writing_prompt)])
    report = report_response.content

    final_report = f"""
    Final Report: 
{'=' * 50}
topic: {message}
{'=' * 50}

{report}
"""

    return {
        "final_report": final_report,
        "next_agent": "supervisor"
    }

In [120]:
def router(state: SupervisorState):
    next_agent = state.get("next_agent", "supervisor")

    if next_agent in ["__end__", "end"]:
        return END
    
    if next_agent in ["supervisor", "web_search", "writer", "arxiv_search"]:
        return next_agent 
    
    return "supervisor"

In [121]:
workflow = StateGraph(SupervisorState)
workflow.add_node("supervisor", supervisor_agent)
workflow.add_node("web_search", web_agent)
workflow.add_node("arxiv_search", arxiv_agent)
workflow.add_node("writer", writer_agent)

workflow.set_entry_point("supervisor")

for node in ["supervisor", "web_search", "arxiv_search", "writer"]:
    workflow.add_conditional_edges(
        node,
        router,
        {
            "supervisor": "supervisor",
            "web_search": "web_search",
            "arxiv_search": "arxiv_search",
            "writer": "writer",
            "__end__": END
        }
    )

In [122]:
graph = workflow.compile()

In [123]:
response = graph.invoke({
    "messages": [HumanMessage(content = query_input)]
})

In [124]:
print(response)

{'messages': [HumanMessage(content='what is attention is all youu need in nlp?', additional_kwargs={}, response_metadata={}, id='03543f1c-e923-4845-a397-1aa71878d786')], 'next_agent': '__end__', 'web_data': ['## The Paper that changed AI Forever. The 2017 paper *“Attention Is All You Need”* introduced the Transformer model, which has revolutionized Natural Language Processing (NLP) and machine learning as a whole. It replaced traditional models that relied heavily on recurrence and convolution. In this blog, I will break down the key ideas from the paper into digestible parts, offering a simplified summary while retaining a technical touch. Before this paper, most NLP models were based on Recurrent Neural Networks (RNNs) or Long Short-Term Memory (LSTM) networks. * **Limited parallelism**: RNNs process data sequentially, which makes them slow for long sequences. The authors of *“Attention Is All You Need”* proposed the **Transformer**, a model that relies entirely on the mechanism of *

In [125]:
print(response["final_report"])


    Final Report: 
topic: [HumanMessage(content='what is attention is all youu need in nlp?', additional_kwargs={}, response_metadata={}, id='03543f1c-e923-4845-a397-1aa71878d786')]

"Attention Is All You Need" refers to a 2017 paper that introduced the Transformer model, which revolutionized Natural Language Processing (NLP) and machine learning. The paper proposed a model that relies entirely on the mechanism of attention, without using any recurrence or convolution, to efficiently handle long-range dependencies and process input in parallel. The idea is that attention, which allows the model to dynamically focus on different parts of the input sequence, is all you need to build powerful sequence processing models. In other words, the paper argues that attention mechanisms can replace traditional recurrent neural networks (RNNs) and convolutional neural networks (CNNs) in NLP tasks, such as language translation, and achieve better results with less computational complexity.

